In [1]:
# python libraries
import mpld3
import numpy as np
from matplotlib import pyplot as plt
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    PidController,
    Simulator,
    SymbolicVectorSystem,
    Variable,
    VectorLogSink,
    sin,
    FixedInputPortValue
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.robot.test_reflected_inertia import (
    TestSimplePendulumWithGearbox,
)
from manipulation.utils import RenderDiagram

# enable mpld3 notebook
if running_as_notebook:
    mpld3.enable_notebook()

In [2]:
def calculate_state(x, u, p):
    q    = x[0]
    qdot = x[1]
    tau  = u[0]
    return [
        qdot,
        (p["N"] * tau - p["m"] * p["g"] * p["l"] * sin(q)) / (p["m"] * p["l"]**2 + p["N"]**2 * p["I_m"])
    ]

In [3]:
p = {
    "N": 160,
    "I_m": 3.46e-4,
    "m": 1.0,  # kg
    "g": 9.81,  # m / s^2
    "l": 0.5,  # m
}

In [4]:
# q_d = 0
# visualize = True

def build(q_d, pendulum_params, gains, visualize=False):
    x = [Variable("theta"), Variable("thetadot")]
    u = [Variable("tau")]
    system = SymbolicVectorSystem(
        input=u, 
        state=x, 
        output=x, 
        dynamics=calculate_state(x, u, pendulum_params))
    
    kp, ki, kd = gains
    controller = PidController([kp], [ki], [kd])

    builder = DiagramBuilder()
    builder.AddSystem(system)
    builder.AddSystem(controller)
    logger = builder.AddSystem(VectorLogSink(2))
    desired_state = builder.AddSystem(ConstantVectorSource([q_d, 0.0]))

    # desired pose -> controller
    builder.Connect(desired_state.get_output_port(0), controller.get_input_port(1))
    # state-action exchange
    builder.Connect(system.get_output_port(0), controller.get_input_port(0))
    builder.Connect(controller.get_output_port(0), system.get_input_port(0))
    # state -> logger
    builder.Connect(system.get_output_port(0), logger.get_input_port(0))

    diagram = builder.Build()
    diagram.set_name("diagram")
    context = diagram.CreateDefaultContext()

    if visualize and running_as_notebook:
        RenderDiagram(diagram, max_depth=1)

    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    context.SetContinuousState([0.0, 0.0, 0.0]) # initialize at zero state

    return simulator, logger.FindLog(context)

In [5]:
# gains = [5, 2, 1]
# simulator, logger = build(0.0, p, gains, visualize=True)

Simulate

In [28]:
# q_d = (5.0 / 8.0) * np.pi
q_d = np.pi
gains = [5, 2, 1]  # [P, I, D] gains.
p["N"] = 10

In [29]:
simulator, logger = build(q_d, p, gains)
simulator.Initialize()
simulator.AdvanceTo(20.0)

time = logger.sample_times()
traj = logger.data()

plt.figure()
plt.plot(time, traj[0, :], "b-")
plt.plot(time, q_d * np.ones(traj.shape[1]), "r--")
plt.xlabel("time (s)")
plt.ylabel("q (rads)")
mpld3.display()

**Answer to the qualitative problem:**

Gearbox makes the system behave more linearly, and higher values of `N` cause gearbox to contribute to a larger portion of the system behavior. Still, not sure why gearbox would make the system behave more linearly. (Maybe it's not linear - maybe it's a sqrt relationship, since the inertia term has quadratic scaling w.r.t. N while torque term has linear scaling. So, maybe it just causes the system to be more "static"?)